In [ ]:
from music21 import *
import pickle
import glob


def parse_mid():
    notes = []
    duration = []

    for midfile in glob.glob('midi_source/*.mid'):

        mid = converter.parse(midfile)
        
        notes_to_parse = None

        parts = instrument.partitionByInstrument(mid)

        if parts:  # file has instrument parts
            notes_to_parse = parts.parts[0].recurse()

        else:  # file has notes in a flat structure
            notes_to_parse = mid.flat.notes
            print('teste')

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                a = str(element.pitch)
                duration.append(str(element.duration.quarterLength))
                a += ("|")
                a += str(element.duration.quarterLength)
                notes.append(a)


            elif isinstance(element, chord.Chord):
                chord_pitches=[]
                

                for p in element.pitches:
                    chord_pitches.append(p.midi)

                
                chord_pitches = str(chord_pitches).strip('[]')
                
                
               
                chord_pitches += ("|")
                
                chord_duration = str(element.duration.quarterLength)
                
                
                chord_pitches += chord_duration
                chord_pitches += ("|chord")

            
                notes.append(chord_pitches)
                with open('data/notes', 'wb') as path:
                    pickle.dump(notes, path)
        

        print(notes)
parse_mid()

['G#3|1/3', '37, 49|4.0|chord', 'C#4|1/3', 'E4|1/3', 'G#3|1/3', 'C#4|1/3', 'E4|1/3', 'G#3|1/3', 'C#4|1/3', 'E4|1/3', 'G#3|1/3', 'C#4|1/3', 'E4|1/3', 'G#3|1/3', '35, 47|4.0|chord', 'C#4|1/3', 'E4|1/3', 'G#3|1/3', 'C#4|1/3', 'E4|1/3', 'G#3|1/3', 'C#4|1/3', 'E4|1/3', 'G#3|1/3', 'C#4|1/3', 'E4|1/3', 'A3|1/3', '45, 33|2.0|chord', 'C#4|1/3', 'E4|1/3', 'A3|1/3', 'C#4|1/3', 'E4|1/3', 'A3|1/3', '30, 42|2.0|chord', 'D4|1/3', 'F#4|1/3', 'A3|1/3', 'D4|1/3', 'F#4|1/3', 'G#3|1/3', '32, 44|2.0|chord', 'C4|1/3', 'F#4|1/3', 'G#3|1/3', 'C#4|1/3', 'E4|1/3', 'G#3|1/3', '44, 32|2.0|chord', 'C#4|1/3', 'E-4|1/3', 'F#3|1/3', 'C4|1/3', 'E-4|1/3', 'E3|1/3', '37, 49, 44|4.0|chord', 'G#3|1/3', 'C#4|1/3', 'G#3|1/3', 'C#4|1/3', 'E4|1/3', 'G#3|1/3', 'C#4|1/3', 'E4|1/3', 'G#4|0.75', 'G#3|1/3', 'C#4|1/3', 'E4|1/3', 'G#4|0.25', 'G#4|3.0', 'G#3|1/3', '36, 44, 48|4.0|chord', 'E-4|1/3', 'F#4|1/3', 'G#3|1/3', 'E-4|1/3', 'F#4|1/3', 'G#3|1/3', 'E-4|1/3', 'F#4|1/3', 'G#4|0.75', 'G#3|1/3', 'E-4|1/3', 'F#4|1/3', 'G#4|0.25', '68